<a href="https://colab.research.google.com/github/eugeniusz-raspberrypi/Export_model_cpk_to_onnx/blob/main/Export_model_ckpt_to_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Instalacja zależności (wykonaj raz)

In [19]:
# KOMÓRKA 0 – Instalacja zależności (wykonaj raz)
!pip install -q piper-tts onnx onnxruntime torch soundfile numpy tqdm
# (piper-tts już zawiera wszystkie potrzebne części train/*)

1. Wgranie Twojego modelu .cpk i config.json

In [40]:
# KOMÓRKA 1 – Wgranie Twojego modelu .cpk + config.yaml + hparams.yaml
from google.colab import drive
drive.mount('/content/drive')

# ZMIEŃ ŚCIEŻKI NA SWOJE (prawy przycisk na plikach → "Kopiuj ścieżkę")
!cp "/content/drive/MyDrive/Piper/epoch=4940-step=7998.ckpt" ./
!cp "/content/drive/MyDrive/Piper/config.yaml" ./
!cp "/content/drive/MyDrive/Piper/hparams.yaml" ./  # jeśli masz

# Sprawdzenie
!ls -lh *.ckpt *.yaml || echo "Brakuje pliku – sprawdź ścieżki!"
print("Pliki wczytane z Drive!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat '/content/drive/MyDrive/Piper/epoch=4940-step=7998.ckpt': No such file or directory
cp: cannot stat '/content/drive/MyDrive/Piper/config.yaml': No such file or directory
cp: cannot stat '/content/drive/MyDrive/Piper/hparams.yaml': No such file or directory
-rw-r--r-- 1 root root 2.6K Dec  9 10:50  config.yaml
-rw-r--r-- 1 root root 366M Dec  9 10:50 'epoch=4940-step=7998.ckpt'
-rw-r--r-- 1 root root  852 Dec  9 10:50  hparams.yaml
Pliki wczytane z Drive!


2. Sam skrypt (wersja Colab-friendly) – wystarczy uruchomić raz i wpisać nazwy plików

In [34]:
# KOMÓRKA 2 – PEŁNY EKSPORT Z OFICJALNEGO NOTEBOOKA (OHF-Voice + TRAINING.md 2025)

# FINALNY FIX - EKSPORT PIPER → ONNX (Colab 2025)
import os
import json
import yaml
import subprocess

# ←←← ZMIEŃ TYLKO TE DWA ←←←
ckpt_file = "epoch=4940-step=7998.ckpt"
output_name = "eugeniusz"
# ←←←←←←←←←←←←←←←←←←←←←←←←←

print(f"🚀 Eksport: {ckpt_file} → {output_name}.onnx\n")

# 1. espeak-ng
print("📦 Instaluję espeak-ng...")
!apt update -qq 2>&1 | grep -v "stable CLI" && apt install -qq espeak-ng libespeak-ng-dev espeak-ng-data -y 2>&1 | tail -3

# 2. Zależności
print("\n📦 Instaluję PyTorch i zależności...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q "lightning>=2.0.0" librosa onnxruntime onnx "numpy>=1.19.0" cython pyyaml

# 3. Klon repo
if not os.path.isdir("piper1-gpl"):
    print("\n📥 Klonuję piper1-gpl...")
    !git clone -q https://github.com/OHF-Voice/piper1-gpl.git
    print("✅ Sklonowano")
else:
    print("\n✅ Repo piper1-gpl już istnieje")

# 4. Instalacja jako pakiet (to rozwiąże relative imports!)
print("\n📦 Instaluję piper jako pakiet...")
os.chdir("/content/piper1-gpl")
!pip install -q -e .
os.chdir("/content")

# 5. Build monotonic_align
print("\n🔨 Buduję monotonic_align...")
os.chdir("/content/piper1-gpl")
!bash build_monotonic_align.sh 2>&1 | tail -5
os.chdir("/content")

# 6. config.yaml → config.json
if not os.path.exists("config.json") and os.path.exists("config.yaml"):
    print("\n🔧 Konwertuję config.yaml → config.json...")
    with open("config.yaml", "r") as f:
        cfg = yaml.safe_load(f)
    with open("config.json", "w") as f:
        json.dump(cfg, f, indent=4)
    print("✅ config.json utworzony")
elif os.path.exists("config.json"):
    print("\n✅ config.json już istnieje")
else:
    print("\n⚠️  OSTRZEŻENIE: Brak config.yaml i config.json")

# 7. EKSPORT - uruchom jako moduł Pythona (-m)
print(f"\n🎯 Eksportuję {ckpt_file} do ONNX...")
print("⏳ To może potrwać 30-90 sekund...\n")

# KLUCZOWA ZMIANA: używamy python -m piper.train.export_onnx
result = subprocess.run(
    [
        "python", "-m", "piper.train.export_onnx",
        f"/content/{ckpt_file}",
        f"/content/{output_name}.onnx"
    ],
    capture_output=True,
    text=True,
    cwd="/content/piper1-gpl"  # Uruchom z katalogu repo
)

# Pokaż output
if result.stdout:
    print(result.stdout)
if result.stderr:
    print("⚠️  Stderr:", result.stderr)

if result.returncode != 0:
    print(f"\n❌ BŁĄD podczas eksportu (kod: {result.returncode})")
    print("\n🔍 Diagnostyka:")
    print(f"   • Checkpoint istnieje: {os.path.exists(f'/content/{ckpt_file}')}")
    print(f"   • config.json istnieje: {os.path.exists('/content/config.json')}")
    print("\n📋 Sprawdź czy plik checkpoint jest prawidłowy:")
    !file /content/{ckpt_file}
    print("\n💡 ROZWIĄZANIE: Upewnij się że:")
    print("   1. Plik .ckpt jest w /content/")
    print("   2. config.json jest w /content/")
    print("   3. Checkpoint pochodzi z tego samego treningu co config")
else:
    print("✅ Eksport zakończony pomyślnie!")

# 8. Tworzenie .onnx.json
if os.path.exists(f"/content/{output_name}.onnx"):
    print(f"\n📝 Tworzę metadane {output_name}.onnx.json...")

    if os.path.exists("/content/config.json"):
        try:
            with open("/content/config.json", "r") as f:
                data = json.load(f)

            # Dodaj inference settings jeśli brak
            if "inference" not in data:
                data["inference"] = {
                    "noise_scale": 0.667,
                    "length_scale": 1.0,
                    "noise_w": 0.8
                }

            data["name"] = output_name
            data["key"] = output_name

            with open(f"/content/{output_name}.onnx.json", "w") as f:
                json.dump(data, f, indent=4)
            print("✅ Metadane zapisane")

        except (json.JSONDecodeError, Exception) as e:
            print(f"⚠️  Nie można odczytać config.json: {e}")
            print("   Tworzę minimalny .onnx.json...")
            minimal = {
                "name": output_name,
                "key": output_name,
                "inference": {
                    "noise_scale": 0.667,
                    "length_scale": 1.0,
                    "noise_w": 0.8
                }
            }
            with open(f"/content/{output_name}.onnx.json", "w") as f:
                json.dump(minimal, f, indent=4)
            print("✅ Minimalny .onnx.json utworzony")
    else:
        print("⚠️  Brak config.json - pomijam tworzenie metadanych")

# 9. Weryfikacja i pobieranie
print("\n" + "="*60)
if os.path.exists(f"/content/{output_name}.onnx"):
    size_mb = os.path.getsize(f"/content/{output_name}.onnx") / 1024 / 1024
    print(f"✅ SUKCES! Model gotowy ({size_mb:.1f} MB)")
    print(f"📁 Plik: {output_name}.onnx")

    if os.path.exists(f"/content/{output_name}.onnx.json"):
        json_size = os.path.getsize(f"/content/{output_name}.onnx.json") / 1024
        print(f"📁 Metadane: {output_name}.onnx.json ({json_size:.1f} KB)")

    # Automatyczne pobieranie
    print("\n⬇️  Pobieram pliki...")
    from google.colab import files
    files.download(f"/content/{output_name}.onnx")
    if os.path.exists(f"/content/{output_name}.onnx.json"):
        files.download(f"/content/{output_name}.onnx.json")

    print("\n🎉 GOTOWE! Model można teraz użyć w Piper TTS")
    print("   Przykład użycia:")
    print(f'   echo "Witaj świecie" | piper -m {output_name}.onnx -c {output_name}.onnx.json -f output.wav')
else:
    print("❌ BŁĄD: Plik .onnx nie został utworzony")
    print("\n🔍 Pliki w /content:")
    !ls -lh /content/*.ckpt /content/*.json /content/*.onnx 2>/dev/null || echo "   Brak oczekiwanych plików"

    print("\n💡 SUGESTIE:")
    print("   1. Sprawdź czy checkpoint nie jest uszkodzony")
    print("   2. Upewnij się że config.json pasuje do modelu")
    print("   3. Zobacz logi powyżej - może tam jest więcej szczegółów")

print("="*60)

🚀 Eksport: epoch=4940-step=7998.ckpt → eugeniusz.onnx

📦 Instaluję espeak-ng...


70 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
libespeak-ng-dev is already the newest version (1.50+dfsg-10ubuntu0.1).
espeak-ng is already the newest version (1.50+dfsg-10ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 70 not upgraded.

📦 Instaluję PyTorch i zależności...

✅ Repo piper1-gpl już istnieje

📦 Instaluję piper jako pakiet...
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for piper-tts (pyproject.toml) ... done

🔨 Buduję monotonic_align...
Possible solutions:
	1. Declare 'maximum_path_each' as 'no

3. (opcjonalna) szybki test czy model działa

In [42]:
import shutil
import os

# Jeśli plik jest na Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ZNAJDŹ pełną ścieżkę do oryginalnego pliku
source_path = "/content/checkpoints/epoch=4940-step=7998.ckpt"  # ZMIEŃ NA SWOJĄ ŚCIEŻKĘ

# Sprawdź rozmiar oryginału
if os.path.exists(source_path):
    orig_size = os.path.getsize(source_path) / (1024 * 1024)
    print(f"📁 Oryginalny plik: {orig_size:.2f} MB")

    # SKOPIUJ z weryfikacją
    print("\n📋 Kopiuję plik (może potrwać 1-2 min)...")
    shutil.copy2(source_path, "/content/epoch=4940-step=7998.ckpt")

    # Weryfikacja
    new_size = os.path.getsize("/content/epoch=4940-step=7998.ckpt") / (1024 * 1024)
    print(f"✅ Skopiowano: {new_size:.2f} MB")

    if abs(orig_size - new_size) < 1:  # Różnica < 1 MB = OK
        print("✅ Rozmiary się zgadzają!")

        # Teraz spróbuj załadować
        import torch
        print("\n🔍 Ładuję checkpoint...")
        ckpt = torch.load("/content/epoch=4940-step=7998.ckpt", map_location='cpu')
        print("✅ Checkpoint załadowany pomyślnie!")
        print(f"\n📋 Klucze: {list(ckpt.keys())}")
    else:
        print(f"⚠️ Rozmiary różnią się: {orig_size:.2f} vs {new_size:.2f} MB")
else:
    print(f"❌ Nie znaleziono pliku: {source_path}")
    print("\n📂 Sprawdź ścieżkę. Możliwe lokacje:")
    !find /content/drive -name "epoch=4940-step=7998.ckpt" 2>/dev/null

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 Oryginalny plik: 806.70 MB

📋 Kopiuję plik (może potrwać 1-2 min)...
✅ Skopiowano: 806.70 MB
✅ Rozmiary się zgadzają!

🔍 Ładuję checkpoint...
✅ Checkpoint załadowany pomyślnie!

📋 Klucze: ['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters']


In [35]:
# KOMÓRKA 3 (opcjonalna) – szybki test czy model działa

# Zainstaluj binarkę piper (najnowszą)
!wget -q https://github.com/rhasspy/piper/releases/download/v1.2.0/piper_amd64.tar.gz
!tar -xf piper_amd64.tar.gz
!chmod +x piper/piper

# Test
!./piper/piper --model {output_name}.onnx --output_file test.wav --text "Cześć, to jest mój sklonowany głos w języku polskim. Działa idealnie!"
files.download("test.wav")

terminate called after throwing an instance of 'std::runtime_error'
  what():  Model file doesn't exist


FileNotFoundError: Cannot find file: test.wav